In [1]:
##!pip install --upgrade git+https://github.com/yhilpisch/tpqoa.git

In [2]:
import os
import time
import numpy as np
import pandas as pd
from pprint import pprint
from pylab import plt, mpl
import tpqoa
plt.style.use('seaborn')
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['font.family'] = 'serif'
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.float_format', '{:.5f}'.format)
np.set_printoptions(suppress=True, precision=4)
os.environ['PYTHONHASHSEED'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'

In [3]:
path = '/Users/dylanyap/working_file/'
api = tpqoa.tpqoa(path + 'crudential.cfg')
ins = api.get_instruments()
ins[:5]

[('AUD/CAD', 'AUD_CAD'),
 ('AUD/CHF', 'AUD_CHF'),
 ('AUD/HKD', 'AUD_HKD'),
 ('AUD/JPY', 'AUD_JPY'),
 ('AUD/NZD', 'AUD_NZD')]

In [4]:
raw = api.get_history(instrument='GBP_USD',
 start='2010.01.01',
 end='2022.12.25',
 granularity='M5',
 price='M')

In [5]:
raw.shape

(970182, 6)

In [6]:
raw.head()

,o,h,l,c,volume,complete
time,,,,,,
2010-01-03 18:10:00,1.61592,1.61592,1.61578,1.61592,3,True
2010-01-03 18:20:00,1.61601,1.61685,1.61567,1.61685,3,True
2010-01-03 18:25:00,1.61635,1.61635,1.61575,1.61635,4,True
2010-01-03 18:30:00,1.61560,1.61635,1.61535,1.61610,5,True
2010-01-03 18:35:00,1.61635,1.61635,1.61635,1.61635,1,True


In [7]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools

/opt/anaconda3/lib/python3.9/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [33]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

In [34]:
TRAIN_START = '2010-01-01'
TRAIN_END = '2022-10-01'
TRADE_START = '2022-10-01'
TRADE_END = '2022-12-25'

In [10]:
INDICATORS = ['macd',
              'rsi_30',
              'cci_30',
              'dx_30',
              'close_30_sma',
              'close_60_sma',
              'boll_ub',
              'boll_lb']

In [11]:
raw = raw.reset_index()
raw.columns = ["date", "open", "high", "low", "close", "volume", "tic"]

In [12]:
raw

,date,open,high,low,close,volume,tic
0,2010-01-03 18:10:00,1.61592,1.61592,1.61578,1.61592,3,True
1,2010-01-03 18:20:00,1.61601,1.61685,1.61567,1.61685,3,True
2,2010-01-03 18:25:00,1.61635,1.61635,1.61575,1.61635,4,True
3,2010-01-03 18:30:00,1.61560,1.61635,1.61535,1.61610,5,True
4,2010-01-03 18:35:00,1.61635,1.61635,1.61635,1.61635,1,True
...,...,...,...,...,...,...,...
970177,2022-12-23 21:35:00,1.20490,1.20523,1.20486,1.20508,315,True
970178,2022-12-23 21:40:00,1.20504,1.20506,1.20477,1.20478,395,True
970179,2022-12-23 21:45:00,1.20478,1.20502,1.20465,1.20496,599,True
970180,2022-12-23 21:50:00,1.20496,1.20518,1.20478,1.20512,607,True


In [13]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(raw)
processed = processed.copy()
processed = processed.fillna(0)
processed = processed.replace(np.inf,0)

/opt/anaconda3/lib/python3.9/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/opt/anaconda3/lib/python3.9/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/opt/anaconda3/lib/python3.9/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
/opt/anaconda3/lib/python3.9/site-packages/finrl/meta/preprocessor/preprocessors.py:158: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

Successfully added technical indicators
Successfully added turbulence index


In [14]:
processed.head()

,date,open,high,low,close,volume,tic,macd,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,boll_ub,boll_lb,turbulence
0,2010-01-03 18:10:00,1.61592,1.61592,1.61578,1.61592,3,True,0.00000,100.00000,66.66667,100.00000,1.61592,1.61592,1.61770,1.61507,0.00000
1,2010-01-03 18:20:00,1.61601,1.61685,1.61567,1.61685,3,True,0.00002,100.00000,66.66667,100.00000,1.61638,1.61638,1.61770,1.61507,0.00000
2,2010-01-03 18:25:00,1.61635,1.61635,1.61575,1.61635,4,True,0.00001,64.26019,-3.37079,100.00000,1.61637,1.61637,1.61730,1.61544,0.00000
3,2010-01-03 18:30:00,1.61560,1.61635,1.61535,1.61610,5,True,-0.00000,54.23436,-56.66667,34.09514,1.61631,1.61631,1.61711,1.61550,0.00000
4,2010-01-03 18:35:00,1.61635,1.61635,1.61635,1.61635,1,True,-0.00000,60.59439,65.60284,34.09514,1.61631,1.61631,1.61701,1.61561,0.00000


In [15]:
train = data_split(processed, TRAIN_START,TRAIN_END)
trade = data_split(processed, TRADE_START,TRADE_END)
print(len(train))
print(len(trade))

952904
17278


In [16]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 1, State Space: 11


In [35]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)

In [36]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


In [19]:
agent = DRLAgent(env = env_train)

if_using_a2c = False
if_using_ddpg = False
if_using_ppo = False
if_using_td3 = False
if_using_sac = True

In [37]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 100000,
    "learning_rate": 0.0001,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

if if_using_sac:
  # set up logger
  tmp_path = RESULTS_DIR + '/sac'
  new_logger_sac = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_sac.set_logger(new_logger_sac)

{'batch_size': 128, 'buffer_size': 100000, 'learning_rate': 0.0001, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cpu device
Logging to results/sac


In [ ]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=100000) if if_using_sac else None

In [22]:
data_risk_indicator = processed[(processed.date<TRAIN_END) & (processed.date>=TRAIN_START)]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

In [23]:
insample_risk_indicator.turbulence.describe()

count   952904.00000
mean         1.18676
std         19.64013
min          0.00000
25%          0.03866
50%          0.21841
75%          0.89558
max      17375.80171
Name: turbulence, dtype: float64

In [24]:
insample_risk_indicator.turbulence.quantile(0.996)

21.428691379754923

In [25]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70, **env_kwargs)

In [26]:
trade.head()

,date,open,high,low,close,volume,tic,macd,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,boll_ub,boll_lb,turbulence
0,2022-10-02 21:00:00,1.11700,1.11700,1.11691,1.11691,2,True,0.00021,56.47710,134.17279,30.35166,1.11597,1.11467,1.11719,1.11440,0.00370
1,2022-10-02 21:05:00,1.11703,1.11706,1.11692,1.11692,9,True,0.00027,56.50526,125.63667,31.17305,1.11599,1.11477,1.11731,1.11436,0.00004
2,2022-10-02 21:10:00,1.11699,1.11776,1.11643,1.11776,55,True,0.00039,58.82029,151.33477,39.99800,1.11605,1.11488,1.11764,1.11426,0.52150
3,2022-10-02 21:15:00,1.11774,1.11804,1.11724,1.11775,92,True,0.00047,58.78176,176.68899,43.11663,1.11608,1.11500,1.11792,1.11423,0.00025
4,2022-10-02 21:20:00,1.11774,1.11797,1.11772,1.11795,27,True,0.00055,59.33292,178.86370,43.11663,1.11613,1.11511,1.11813,1.11413,0.02736


In [27]:
trained_model = trained_sac
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_model, 
    environment = e_trade_gym)

hit end!


In [28]:
df_account_value.shape

(17278, 2)

In [29]:
df_account_value.tail()

,date,account_value
17273,2022-12-23 21:35:00,1000000.00000
17274,2022-12-23 21:40:00,1000000.00000
17275,2022-12-23 21:45:00,1000000.00000
17276,2022-12-23 21:50:00,1000000.00000
17277,2022-12-23 21:55:00,1000000.00000


In [30]:
df_actions.head()

,date,actions
0,2022-10-02 21:00:00,[0]
1,2022-10-02 21:05:00,[0]
2,2022-10-02 21:10:00,[0]
3,2022-10-02 21:15:00,[0]
4,2022-10-02 21:20:00,[0]


In [31]:
len(df_actions)

17277

In [32]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

==============Get Backtest Results===========
Annual return         0.00000
Cumulative returns    0.00000
Annual volatility     0.00000
Sharpe ratio              NaN
Calmar ratio              NaN
Stability             0.00000
Max drawdown          0.00000
Omega ratio               NaN
Sortino ratio             NaN
Skew                      NaN
Kurtosis                  NaN
Tail ratio                NaN
Daily value at risk   0.00000
dtype: float64


/opt/anaconda3/lib/python3.9/site-packages/pyfolio/timeseries.py:724: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  stats = pd.Series()
/opt/anaconda3/lib/python3.9/site-packages/empyrical/stats.py:710: RuntimeWarning: invalid value encountered in divide
  np.divide(
/opt/anaconda3/lib/python3.9/site-packages/empyrical/stats.py:799: RuntimeWarning: invalid value encountered in divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
/opt/anaconda3/lib/python3.9/site-packages/empyrical/stats.py:1527: RuntimeWarning: invalid value encountered in scalar divide
  return np.abs(np.percentile(returns, 95)) / \
